In [1]:
import pandas as pd
from pathlib import Path
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
from pandarallel import pandarallel

In [15]:
pandarallel.initialize(nb_workers=10, progress_bar=True)

INFO: Pandarallel will run on 10 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.


In [24]:
analyzer = SentimentIntensityAnalyzer()

In [4]:
src = "../../data/"

In [27]:
fname = "combined_US_politician_twitter_timelines_2010-11-06_to_2023-02-11_clean.csv.gzip"
texts = pd.read_csv(
    Path(src, "tweets", fname),
    compression="gzip",
    dtype={"id":str, "author_id":str},
    usecols=["id", "text", "author_id"]
)

In [28]:
def calculate_VADER(text):
    vs = analyzer.polarity_scores(text)
    return str(vs["neg"]) + "+" + str(vs["neu"]) + "+" + str(vs["pos"]) + "+" + str(vs["compound"])

In [29]:
texts["VADER_raw"] = texts["text"].parallel_apply(calculate_VADER)

In [30]:
texts["neg"] = texts["VADER_raw"].parallel_apply(lambda x: float(x.split("+")[0]))

In [31]:
texts["neu"] = texts["VADER_raw"].parallel_apply(lambda x: float(x.split("+")[1]))

In [32]:
texts["pos"] = texts["VADER_raw"].parallel_apply(lambda x: float(x.split("+")[2]))

In [33]:
texts["compound"] = texts["VADER_raw"].parallel_apply(lambda x: float(x.split("+")[3]))

In [34]:
cols = ["id", "neg", "neu", "pos", "compound"]
fname = "combined_US_politician_twitter_timelines_2010-11-06_to_2022-12-31_clean_VADER.csv.gzip"
texts[cols].to_csv(
    Path(src, "tweets", fname),
    compression="gzip",
    index=False
)